**--- Part One ---**

In [1]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
# case = "jfm"
case = "gatton"

# verbose output
verbose = False

import numpy as np
from itertools import cycle

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

import numpy as np

abc = {val:idx for idx,val in enumerate('abcdefghijklmnopqrstuvwxyz')}
abc['S'] = -1
abc['E'] = 26

class Elf():
    def __init__(self,pos):
        self.pos = np.array(pos)

class Grid():
    def __init__(self, start=None):
        self.grid = np.array([[abc[val] for val in row.strip()] for row in input_lines])
        if start is not None:
            self.start = np.array(start)
        else:
            tmp = np.where(self.grid==-1)
            self.start = np.array((tmp[0][0], tmp[1][0]))
        self.elves = [Elf(self.start)]
        tmp = np.where(self.grid==26)
        self.dest = np.array((tmp[0][0], tmp[1][0]))
        self.hist = set()
        self.width = len(self.grid[0])
        self.height = len(self.grid)
        self.store_pos(self.start)
        self.round = 0
        
    @property
    def finished(self):
        if tuple(self.dest) in self.hist:
            print(f'finished: {self.round}')
            return True
        for e in self.elves:
            if e.pos[0]==self.dest[0] and e.pos[1]==self.dest[1]:
                return True
        else:
            return False
        #return any([np.all(self.dest==e.pos) for e in self.elves])
    
    def move_options(self,elf):
        options = []
        for i in ((0,1),(0,-1),(1,0),(-1,0)):
            next = elf.pos + i
            if next[0]<0 or next[1]<0 or next[0]>=self.height or next[1]>=self.width:
                continue
            val = self.grid[tuple(elf.pos)]
            if tuple(next) not in self.hist:
                # print(f'{next=} val={self.grid[tuple(next)]}')
                if self.grid[tuple(next)]<=val+1:
                    # print(f'adding: {next}')
                    options.append(next)
        return options
    
    def next_move(self):
        new_elves = []
        for elf in self.elves:
            for option in self.move_options(elf):
                new_elves.append(Elf(option))
                self.store_pos(option)
        self.elves = new_elves
        self.round += 1

    def execute_continuous(self,max_rounds=None):
        if max_rounds is not None:
            n = 0
            while (not self.finished) and (n<=max_rounds+1) and (len(self.elves)>0):
                self.next_move()
                n += 1
        else:
            print('executing')
            while (not self.finished) and len(self.elves)>0:
                self.next_move()
                # print(self.round)

    def store_pos(self, pos):
        self.hist.add(tuple(pos))

In [2]:
grid = Grid()
grid.execute_continuous()
print(grid.finished)
print(grid.round)

executing
finished: 352
finished: 352
True
352


**--- Part Two ---**

In [17]:
grid = Grid()
starting_points = []
distances = []
tmp = np.where(grid.grid == 0)
for i,v in enumerate(tmp[0]):
    starting_points.append([tmp[0][i],tmp[1][i]])
sorted_points = sorted(starting_points, key=lambda x: abs(x[0]-grid.dest[0]) + abs(x[1]-grid.dest[1]))
max_rounds = None
for point in sorted_points:
    # print(f'checking {point}')
    grid = Grid(point)
    grid.execute_continuous(max_rounds)
    if grid.finished: distances.append(grid.round)
    if len(distances)>0: max_rounds = min(distances)
min(distances)                                                                                                      

386